In [1]:
%pylab inline

import spacy
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud, ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn
from unidecode import unidecode
from unicodedata import normalize
import statsmodels.api as sm

# Define default stopwords list
nlp = spacy.load('pt_core_news_sm')
stoplist = spacy.lang.pt.stop_words.STOP_WORDS
punctuations = string.punctuation

# Let's using Nate's Silver 538 style for graphs
plt.style.use('fivethirtyeight')

# Load data from local environment
df_extracted_essays = pd.read_csv('df_extracted_essays.csv')

# Basic information about the files
print(f'Qty Articles: {df_extracted_essays.shape[0]}')
print(f'Qty Distinct Authors: {len(df_extracted_essays.author.unique())}')

Populating the interactive namespace from numpy and matplotlib
Qty Articles: 2860
Qty Distinct Authors: 495


In [2]:
df_extracted_essays.head(10)

,author,url,date_transform,date,text_processed
0,Jeffrey Tucker,https://mises.org.br/Article.aspx?id=17,"['', '24', 'jan', '2008']",2008-01-24,propriedade intelectual chave sucesso maiores ...
1,Walter Block,https://mises.org.br/Article.aspx?id=19,"['', '30', 'jan', '2008']",2008-01-30,economista feliz 0 0 44 320 tdblockquotemurray...
2,Christopher Westley,https://mises.org.br/Article.aspx?id=22,"['', '30', 'jan', '2008']",2008-01-30,loucuras fiscais chegamos anos gastanca inflac...
3,Ron Paul,https://mises.org.br/Article.aspx?id=23,"['', '30', 'jan', '2008']",2008-01-30,crise imobiliaria politica monetaria federal r...
4,Jim Bradley,https://mises.org.br/Article.aspx?id=24,"['', '30', 'jan', '2008']",2008-01-30,culpem emprestadores subprime atual retorica c...
5,Lew Rockwell,https://mises.org.br/Article.aspx?id=25,"['', '30', 'jan', '2008']",2008-01-30,contratos governamentais versus negocios verda...
6,Jeffrey Tucker,https://mises.org.br/Article.aspx?id=26,"['', '30', 'jan', '2008']",2008-01-30,quao livre livre mercado consegue perceber err...
7,Gary Danelishen,https://mises.org.br/Article.aspx?id=27,"['', '30', 'jan', '2008']",2008-01-30,economia austriaca vs economia bernanke econom...
8,Ron Paul,https://mises.org.br/Article.aspx?id=28,"['', '30', 'jan', '2008']",2008-01-30,colapso enron perante comite servicos financei...
9,Art Carden,https://mises.org.br/Article.aspx?id=30,"['', '30', 'jan', '2008']",2008-01-30,expansao credito investimentos improdutivos co...


In [3]:
df_extracted_essays['text_processed'].head(1)[0]

'propriedade intelectual chave sucesso maiores tragedias referentes leis propriedade intelectual geram confusao empresarios sucedidos impressao apesar verdade fortuna direito autoral marca registrada patente necessariamente avaliacao negocios razao defendem propriedade intelectual fosse verdadeira forca vital negocios deixam credito principal motivo propria criatividade disposicao assumir riscos acoes geralmente baseadas mercado frequentemente julgamento empiricamente incorreto deles gera tragedia credito realizacoes verdade devidas proprias atividades empreendedoras fato faltam historias prontas conduzem falsa impressao inumeros relatos negocios mencionam lucros surgem patentes delas assumem causal suposicao historia espirito empreendedor deixa historia heroica riscos recompensas torna historia decisoes escriturarios patentes advogados direitos autorais resultado muitas pessoas acham razao eua terem crescido rapidamente seculo xix protecao propriedade intelectual assumem protecao idei

In [4]:
def get_lexical_diversity(df, author):
    dataframe = df[df['author'] == author]

    # Word stats
    full_text_count = pd.DataFrame(Counter(" ".join(dataframe["text_processed"]).split()), index=[0])
    full_text_count = full_text_count.T
    full_text_count = full_text_count.reset_index()
    full_text_count.columns = ['word', 'qty']

    # Distinct words to include in numerator
    distinct_words = set()
    dataframe['text_processed'].str.lower().str.split().apply(distinct_words.update)
    total_distinct_words = len(distinct_words)

    # All words to include in denominator
    total_words = full_text_count['qty'].sum()

    lexical_diversity = round((total_distinct_words / total_words),2)

    print(f'Lexical Diversity for {author}: {lexical_diversity}')

In [5]:
authors = ['Leandro Roque',
          'Lew Rockwell',
          'Jeffrey Tucker',
          'Fernando Ulrich',
           'Ludwig von Mises',
           'Ubiratan Jorge Iorio',
           'Gary North',
           'Murray N. Rothbard',
           'Helio Beltrão',
           'Juan Ramón Rallo',
           'Hans-Hermann Hoppe',
           'Fernando Chiocca',
           'Rodrigo Constantino',
          ]

for author in authors:
    get_lexical_diversity(df_extracted_essays, author)

Lexical Diversity for Leandro Roque: 0.12
Lexical Diversity for Lew Rockwell: 0.21
Lexical Diversity for Jeffrey Tucker: 0.24
Lexical Diversity for Fernando Ulrich: 0.22
Lexical Diversity for Ludwig von Mises: 0.18
Lexical Diversity for Ubiratan Jorge Iorio: 0.22
Lexical Diversity for Gary North: 0.19
Lexical Diversity for Murray N. Rothbard: 0.16
Lexical Diversity for Helio Beltrão: 0.36
Lexical Diversity for Juan Ramón Rallo: 0.25
Lexical Diversity for Hans-Hermann Hoppe: 0.18
Lexical Diversity for Fernando Chiocca: 0.39
Lexical Diversity for Rodrigo Constantino: 0.46


In [6]:
def get_word_ngrams_list(df, author, word_ngram):

    def get_top_word_n_bigram(corpus, n=None):
        vec = CountVectorizer(ngram_range=(word_ngram, word_ngram)).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:n]


    common_words = get_top_word_n_bigram(df[df['author'] == author]['text_processed'], 20)
    df3 = pd.DataFrame(common_words, columns = ['ngram' , 'qty'])

    return df3

In [7]:
get_word_ngrams_list(df_extracted_essays, 'Leandro Roque', 3)

,ngram,qty
0,quantidade dinheiro economia,104
1,taxa basica juros,43
2,grafico abaixo mostra,37
3,evolucao taxa cambio,32
4,bancario reservas fracionarias,28
5,aumento oferta monetaria,27
6,juros mercado interbancario,27
7,fonte banco central,27
8,junto banco central,26
9,grafico seguir mostra,26


In [8]:
get_word_ngrams_list(df_extracted_essays, 'Lew Rockwell', 3)

,ngram,qty
0,ludwig von mises,31
1,51 51 51,15
2,enus 51 51,13
3,banco central americano,9
4,economia livre mercado,8
5,fannie mae freddie,8
6,mae freddie mac,8
7,durante periodo expansao,7
8,boas conexoes politicas,7
9,governo rotula pratica,7


In [9]:
get_word_ngrams_list(df_extracted_essays, 'Jeffrey Tucker', 3)

,ngram,qty
0,ludwig von mises,9
1,pessoas redor mundo,5
2,condicoes medicas preexistentes,5
3,leis propriedade intelectual,4
4,melhor situacao caso,4
5,maneira estatista pensar,4
6,there will be,4
7,centenas cidades redor,4
8,cidades redor mundo,4
9,governos redor mundo,3


In [10]:
get_word_ngrams_list(df_extracted_essays, 'Fernando Ulrich', 3)

,ngram,qty
0,banco central europeu,14
1,ludwig von mises,13
2,caixa economica federal,12
3,banco central americano,11
4,lei oferta demanda,10
5,emirados arabes unidos,9
6,fractional reserve banking,9
7,bancario reservas fracionarias,9
8,banco nacional suica,9
9,fonte bacen autor,9


In [11]:
get_word_ngrams_list(df_extracted_essays, 'Ludwig von Mises', 3)

,ngram,qty
0,propriedade privada meios,26
1,privada meios producao,26
2,precos fatores producao,13
3,funcionamento economia mercado,13
4,baseada propriedade privada,11
5,padrao vida assalariados,11
6,producao larga escala,10
7,produtividade marginal trabalho,10
8,precos futuros bens,10
9,quantidade dinheiro circulacao,9


In [12]:
get_word_ngrams_list(df_extracted_essays, 'Ubiratan Jorge Iorio', 3)

,ngram,qty
0,escola austriaca economia,28
1,economia mundo real,17
2,teoria austriaca ciclos,15
3,austriaca ciclos economicos,14
4,historia pensamento economico,13
5,condicoes incerteza genuina,12
6,producao bens capital,12
7,economistas escola austriaca,12
8,ludwig von mises,12
9,teoria austriaca capital,11


In [13]:
get_word_ngrams_list(df_extracted_essays, 'Gary North', 3)

,ngram,qty
0,banco central europeu,39
1,banco central americano,17
2,bancario reservas fracionarias,17
3,ludwig von mises,16
4,juros longo prazo,16
5,titulos tesouro americano,16
6,expansao base monetaria,14
7,comprar titulos divida,12
8,taxas juros longo,12
9,uniao monetaria europeia,11


In [14]:
get_word_ngrams_list(df_extracted_essays, 'Murray N. Rothbard', 3)

,ngram,qty
0,ludwig von mises,32
1,industrias bens capital,18
2,times new enus,18
3,federal reserve system,17
4,margin spanop margin,17
5,spanop margin 18pt,16
6,times new spanop,15
7,economia livre mercado,13
8,direito livre expressao,13
9,teoria ciclos economicos,12


In [15]:
get_word_ngrams_list(df_extracted_essays, 'Helio Beltrão', 3)

,ngram,qty
0,ludwig von mises,9
1,jornal folha paulo,9
2,igualdade perante lei,8
3,instituto mises brasil,7
4,ibef news helio,7
5,news helio beltrao,7
6,publicado originalmente jornal,7
7,originalmente jornal folha,7
8,defensor livre mercado,4
9,neutralidade isonomia impostos,4


In [16]:
get_word_ngrams_list(df_extracted_essays, 'Juan Ramón Rallo', 3)

,ngram,qty
0,renda per capita,22
1,mera vontade politica,10
2,deficit orcamentario governo,8
3,reducao gastos governo,8
4,ricacos decada 1980,7
5,bens futuros bens,7
6,politicas precos minimos,6
7,economia jogo soma,5
8,durante ultimas decadas,5
9,divida publica 2013,5


In [17]:
get_word_ngrams_list(df_extracted_essays, 'Hans-Hermann Hoppe', 3)

,ngram,qty
0,ludwig von mises,20
1,problema ordem social,20
2,daily bell hoppe,15
3,desejo propriedade alheia,13
4,bens apropriados produzidos,12
5,apropriacao original propriedade,11
6,original propriedade privada,11
7,direitos propriedade privada,10
8,tomada suprema decisoes,10
9,atos apropriacao original,10


In [18]:
get_word_ngrams_list(df_extracted_essays, 'Fernando Chiocca', 3)

,ngram,qty
0,ludwig von mises,7
1,macaco subia escada,4
2,there is crime,4
3,all his life,4
4,morfeu neo morfeu,3
5,jato agua gelada,3
6,order but order,3
7,but order must,3
8,order must be,3
9,atender demandas consumidores,3


In [19]:
get_word_ngrams_list(df_extracted_essays, 'Rodrigo Constantino', 3)

,ngram,qty
0,reducao taxa juros,3
1,20 milhoes toneladas,3
2,tamanho otimo firma,2
3,eficiente livre mercado,2
4,ruas estradas block,2
5,defender livre mercado,2
6,mecanismos adequados incentivo,2
7,ganham 30 salarios,2
8,salarios paga media,2
9,altruismo esforco alheio,2


In [20]:
# LDA Analysis dataframes
df_text_consolidated_leandro_roque = df_extracted_essays[df_extracted_essays['author'] == 'Leandro Roque']
df_text_consolidated_lew_rockwell = df_extracted_essays[df_extracted_essays['author'] == 'Lew Rockwell']
df_text_consolidated_jeffrey_tucker = df_extracted_essays[df_extracted_essays['author'] == 'Jeffrey Tucker']
df_text_consolidated_fernando_ulrich = df_extracted_essays[df_extracted_essays['author'] == 'Fernando Ulrich']
df_text_consolidated_ludwig_von_mises = df_extracted_essays[df_extracted_essays['author'] == 'Ludwig von Mises']
df_text_consolidated_ubiratan_jorge_iorio = df_extracted_essays[df_extracted_essays['author'] == 'Ubiratan Jorge Iorio']
df_text_consolidated_gary_north = df_extracted_essays[df_extracted_essays['author'] == 'Gary North']
df_text_consolidated_murray_rothbard = df_extracted_essays[df_extracted_essays['author'] == 'Murray N. Rothbard']
df_text_consolidated_helio_beltrao = df_extracted_essays[df_extracted_essays['author'] == 'Helio Beltrão']
df_text_consolidated_juan_ramon_rallo = df_extracted_essays[df_extracted_essays['author'] == 'Juan Ramón Rallo']
df_text_consolidated_hans_hermann_hoppe = df_extracted_essays[df_extracted_essays['author'] == 'Hans-Hermann Hoppe']
df_text_consolidated_fernando_chiocca = df_extracted_essays[df_extracted_essays['author'] == 'Fernando Chiocca']
df_text_consolidated_rodrigo_constantino = df_extracted_essays[df_extracted_essays['author'] == 'Rodrigo Constantino']

In [21]:
def get_topics(df, n_components, number_words):
    
    # Convert to list
    data = df.text_processed.values.tolist()

    # Remove special characters
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    vectorizer = CountVectorizer(analyzer='word',       
                                 stop_words=stoplist,
                                 lowercase=True,
                                )

    data_vectorized = vectorizer.fit_transform(data)

    # Materialize the sparse data
    data_dense = data_vectorized.todense()

    # Compute Sparsicity = Percentage of Non-Zero cells
    print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

    # Build LDA Model
    lda_model = LatentDirichletAllocation(n_components=n_components,
                                          max_iter=10,
                                          learning_method='online',
                                          random_state=42,
                                          batch_size=10,
                                          evaluate_every = -1,
                                          n_jobs=-1,
                                         )
    lda_output = lda_model.fit_transform(data_vectorized)

    # Helper function
    def print_topics(model, count_vectorizer, n_top_words):
        words = count_vectorizer.get_feature_names()
        for topic_idx, topic in enumerate(model.components_):
            print("\nTopic #%d:" % topic_idx)
            print(" ".join([words[i]
                            for i in topic.argsort()[:-n_top_words - 1:-1]]))

    # Print the topics found by the LDA model
    print("Topics found via LDA:")
    lda_model.fit(data_vectorized)

    print_topics(lda_model, vectorizer, number_words)
    
    return lda_model, data_vectorized, data, lda_output, vectorizer

<>:7: DeprecationWarning: invalid escape sequence \S
<>:10: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \S
<>:10: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \S
<>:10: DeprecationWarning: invalid escape sequence \s
<ipython-input-21-c22d93fb011f>:7: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-21-c22d93fb011f>:10: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [22]:
def get_lda_plot(lda_model, data_vectorized, vectorizer):
    pyLDAvis.enable_notebook()
    panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
    return panel


# The warnings it's caused because the pip version.
# This warnings doesn't occur in the conda version.
# ref: https://github.com/bmabey/pyLDAvis/issues/96

In [23]:
lda_model, data_vectorized, data, lda_output, vectorizer = \
    get_topics(df_text_consolidated_leandro_roque, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  2.319272233884546 %
Topics found via LDA:

Topic #0:
dominancia rentenmark compras rentenbank sacolas sacola premissa ida palha trancado

Topic #1:
governo mercado empresas concorrencia livre servicos setor empresa seja pobres

Topic #2:
depressao americana americano decada levy crise fannie freddie fed eua

Topic #3:
rotativo credenciador normal churrascaria emissores mensalao cartoes quitar pagou mexicanos

Topic #4:
tickets desigualdade fundamentos ticket armazem 10g armazens sacar gato ideologicos

Topic #5:
precos dinheiro governo economia banco juros credito bancos central aumento

Topic #6:
gasolina caminhoneiros frete fgc panamericano 107 alcool 00 101 transportadoras

Topic #7:
soro paracetamol leite surrupia retaliar conservadas red chefoes escovar desentendeuse

Topic #8:
is this that it bank central money be government will

Topic #9:
currency board regime emissor conversibilidade argentino atrelado dezembro moedaancora atrelada


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
5      -10.798889  -35.719128       1        1  68.864208
1       27.638935  103.917130       2        1  22.988368
8       97.069420   54.659439       3        1   2.779657
2       55.592319   -8.702127       4        1   2.466107
6      -64.880646   72.378212       5        1   1.367122
9       -0.887719   35.351284       6        1   0.724737
3       47.993011  -92.750084       7        1   0.359910
4      -83.570839  -11.590418       8        1   0.218297
0      121.488457  -39.824509       9        1   0.193628
7      -44.128960 -100.362366      10        1   0.037967, topic_info=      Category        Freq          Term       Total  loglift  logprob
4030   Default  762.000000       central  762.000000  30.0000  30.0000
13796  Default  790.000000       mercado  790.000000  29.0000  29.0000
5992   Default   83.000000      currency   83.000000  28.0000  28.0000
15910  Default  214.000000           pib  214.000000  27.0000  27.0000
12513  Default  103.000000            is  103.000000  26.0000  26.0000
...        ...         ...           ...         ...      ...      ...
2529   Topic10    0.030398     assumirao    0.512700   5.0509  -7.7288
11297  Topic10    0.030396      impactar    0.512744   5.0507  -7.7289
6432   Topic10    0.030395  dentifricios    0.512794   5.0506  -7.7289
3837   Topic10    0.030389        caries    0.512949   5.0501  -7.7291
12898  Topic10    0.065787         leite    4.126359   3.7375  -6.9568

[483 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
0          1  0.175247         00
0          3  0.233662         00
0          5  0.584155         00
21         5  0.931170         05
31         5  0.935362         08
...      ...       ...        ...
21450      2  0.971795  violencia
21684      3  0.999181      which
21694      3  0.969008       will
21700      3  0.977537       with
21717      3  0.987768      would

[544 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 9, 3, 7, 10, 4, 5, 1, 8])

In [24]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_lew_rockwell, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  3.5438874123799162 %
Topics found via LDA:

Topic #0:
mises austriaca teoria left escola economia austriacos anos menger natureza

Topic #1:
censo hospedaria israel jose davi herodes jesus reis recenseamento magos

Topic #2:
esquerda ersidade esquerdista esquerdistas progressista francesa uniformidade igualdade kuehneltleddihn esquerdismo

Topic #3:
stockman aig nuclear ltcm 2012 defenderam socorrida impediram janeiro sp

Topic #4:
presidente 51 enus halliburton federal membros liberalismo iraque estados vicepresidente

Topic #5:
mval domina perdeu longinquo oligopolios concordavam tomo 1937 coordernadora mwrapindent

Topic #6:
argentina energia apagoes peso companhias cavallo argentino calor eletricidade corrida

Topic #7:
mises hoppe rothbard was helio chiocca is brasil beltrao leitor

Topic #8:
remedios google farmacias anuncios genericos farmaceutica spams cartel anunciar generico

Topic #9:
governo mercado pessoas economia liberdade livre sociedade mundo dinheiro send

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
9     -156.158066  -39.793201       1        1  87.705794
0      122.240608   80.012939       2        1   5.237837
4      131.144653  -38.665417       3        1   2.211678
7       43.285500 -118.941185       4        1   1.799767
2       28.744982    1.980243       5        1   1.239710
1     -165.069839   78.882874       6        1   0.799839
6       23.810841  122.853218       7        1   0.500606
3      -77.211029  159.161865       8        1   0.277582
8      -57.723721  -82.610832       9        1   0.214621
5      -62.662018   38.254551      10        1   0.012566, topic_info=      Category        Freq         Term       Total  loglift  logprob
9281   Default  161.000000        mises  161.000000  30.0000  30.0000
5604   Default   83.000000     esquerda   83.000000  29.0000  29.0000
4801   Default  349.000000     economia  349.000000  28.0000  28.0000
6978   Default  678.000000      governo  678.000000  27.0000  27.0000
972    Default  155.000000         anos  155.000000  26.0000  26.0000
...        ...         ...          ...         ...      ...      ...
2912   Topic10    0.000649  concretizou    3.942176   0.2708  -9.4872
11360  Topic10    0.000649    protegido    3.941300   0.2704  -9.4879
6978   Topic10    0.000685      governo  678.599655  -4.8242  -9.4340
6503   Topic10    0.000653          fmi   11.091855  -0.7586  -9.4822
9168   Topic10    0.000667      mercado  469.064833  -4.4813  -9.4604

[524 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
52         1  1.016565     1937
86         2  0.692144     1991
108        1  0.947160     2012
147        3  0.950757       51
147        4  0.052820       51
...      ...       ...      ...
14507      4  0.575300       we
14516      4  0.911312     were
14532      4  0.841611     with
14536      4  0.575318     work
14570      6  1.109163  zelotes

[499 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 1, 5, 8, 3, 2, 7, 4, 9, 6])

In [25]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_jeffrey_tucker, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  3.4193083749374416 %
Topics found via LDA:

Topic #0:
was they but that this it by is be money

Topic #1:
was it they that is money by be but this

Topic #2:
dinheiro economia mundo bitcoin teoria bancos governo mercado banco fed

Topic #3:
maconha inveja carro fumar alheio neonazista olhado porta chaves neonazistas

Topic #4:
patinetes pele patinete eletricas animais eletrica pedestre selvagem cacadores kia

Topic #5:
pessoas mundo mercado governo liberdade vida maneira realmente seja trabalho

Topic #6:
comercio livre fidel facebook dinheiro cubana acordos pessoas paises ideia

Topic #7:
venezuela obamacare fome venezuelano perez comer seguradoras cumana caminhoes inanicao

Topic #8:
destrutivismo intelectual cocacola coringa propriedade cds negocios patentes companhia minimalista

Topic #9:
pianos industria piano comercio tarifas paises harley amp st1personname harleydavidson


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
5      -37.357304   25.624166       1        1  80.095041
2        3.820787 -174.834137       2        1  13.592208
1      134.439087  171.678314       3        1   1.847648
8      -19.694920  164.000351       4        1   1.071190
3      160.879074 -120.370041       5        1   1.035954
7       51.112141  -58.167545       6        1   0.936507
9       79.429169   60.529148       7        1   0.784506
4      201.315750   18.955818       8        1   0.609119
6     -149.951538   76.600212       9        1   0.013914
0     -109.518867  -82.924690      10        1   0.013914, topic_info=      Category        Freq         Term       Total  loglift  logprob
11258  Default   99.000000  propriedade   99.000000  30.0000  30.0000
2710   Default   55.000000     comercio   55.000000  29.0000  29.0000
4543   Default  165.000000     dinheiro  165.000000  28.0000  28.0000
7999   Default   50.000000  intelectual   50.000000  27.0000  27.0000
2234   Default   58.000000        carro   58.000000  26.0000  26.0000
...        ...         ...          ...         ...      ...      ...
10538  Topic10    0.000623       pianos    5.736184  -0.2477  -9.5181
13168  Topic10    0.000618       system    4.164149   0.0644  -9.5263
1882   Topic10    0.000632      bitcoin   49.658475  -2.3918  -9.5038
9092   Topic10    0.000627      mercado  308.689535  -4.2274  -9.5123
7733   Topic10    0.000623    industria   24.175597  -1.6869  -9.5187

[590 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
70         1  0.135237     2008
70         2  0.811424     2008
105        6  1.064605      400
199        4  0.575760  abracou
353        1  0.741604  acordos
...      ...       ...      ...
14415      3  0.801601    world
14436      1  0.856372     york
14436      7  0.142729     york
14442      2  0.910271    yunus
14445      5  0.983293     zelo

[499 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 2, 9, 4, 8, 10, 5, 7, 1])

In [26]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_fernando_ulrich, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  3.3480281297726644 %
Topics found via LDA:

Topic #0:
dubai is monetaria expansao projetos havia mercado 2008 ano mundo

Topic #1:
is not that monetaria by central it banco moeda are

Topic #2:
japoneses fabricas supermercados escritorios reerguera sucessao comunidades edificacoes tsunami macroeconomistas

Topic #3:
trichet vs hayek keynes considerado colega ativar questiona dividaem jesus

Topic #4:
dinheiro bitcoin moeda bancos credito banco ouro bancario mercado economia

Topic #5:
moeda dinheiro precos monetaria mercado economia reservas central dubai bc

Topic #6:
dubai projetos 2008 havia credito eau demanda estavam empresas construcao

Topic #7:
mercado juros economia governo monetaria banco anos precos moeda bancos

Topic #8:
is it banks by are that not this banking be

Topic #9:
bitcoin banco reservas monetaria moeda central politica monetario egipcios dinheiro


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
7      -97.804710   41.685383       1        1  45.333581
4     -201.099152   -5.160827       2        1  35.882642
8     -159.954117 -125.636612       3        1  11.901665
6        4.420489   13.020385       4        1   6.665847
3      -20.203176  135.268661       5        1   0.114460
2     -156.825180  131.835312       6        1   0.048702
9       75.483627  -73.868553       7        1   0.013276
1      -71.348404  -61.247791       8        1   0.013276
0      -18.730877 -157.792389       9        1   0.013276
5       96.640038   69.071777      10        1   0.013276, topic_info=      Category        Freq      Term       Total  loglift  logprob
7420   Default  157.000000        is  157.000000  30.0000  30.0000
4175   Default  309.000000  dinheiro  309.000000  29.0000  29.0000
7447   Default   86.000000        it   86.000000  28.0000  28.0000
1805   Default  230.000000   bitcoin  230.000000  27.0000  27.0000
4369   Default   61.000000     dubai   61.000000  26.0000  26.0000
...        ...         ...       ...         ...      ...      ...
1641   Topic10    0.000631    bancos  338.262824  -4.2655  -9.4570
1510   Topic10    0.000631   aumento   78.140789  -2.8002  -9.4571
6252   Topic10    0.000631  governos   72.256842  -2.7220  -9.4571
2028   Topic10    0.000630  cambiais   20.995223  -1.4869  -9.4579
10109  Topic10    0.000630  projetos   35.901514  -2.0239  -9.4585

[644 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
13         1  0.889861     10
13         2  0.081961     10
13         3  0.035126     10
83         1  0.280750   2006
83         2  0.070188   2006
...      ...       ...    ...
13130      3  0.992416     we
13157      3  0.988220  which
13172      3  0.989222   will
13183      3  0.988763   with
13208      3  0.981476  would

[484 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 9, 7, 4, 3, 10, 2, 1, 6])

In [27]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_ludwig_von_mises, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  4.510914541985678 %
Topics found via LDA:

Topic #0:
governo inflacao precos moeda aumento dinheiro mercado economistas keynes demanda

Topic #1:
mercado keynes precos economia moeda economistas socialismo dinheiro salarios liberdade

Topic #2:
regulamentacoes brechas leite leigo respirar ganancia viola fiscalizadores impingidas soldo

Topic #3:
keynes economistas politicas eram grabretanha inflacionistas professor doutrinas autores ensinamentos

Topic #4:
generos provincias moradores iv provocou haviamse arrendatarios aproveitaram aglomeracoes tabelamento

Topic #5:
credito bancos juros deumer taxa extensao lucratividade bancarias emboom say

Topic #6:
precos governo mercado keynes inflacao capital empresas capitalismo socialismo credito

Topic #7:
doenca seguridade autodeterminacao acidentes doencas pertencer hoppe seguro neurose territorio

Topic #8:
mercado producao pessoas economia governo precos bens capital trabalho fato

Topic #9:
humana logica cooperacao homem soc

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
8      194.280792  145.437714       1        1  88.747986
9      -47.211666 -138.443497       2        1   6.493094
3      199.149994   -8.868906       3        1   2.689013
5     -171.599594  -47.000221       4        1   1.210274
2     -176.462524  107.303818       5        1   0.385240
7      -55.391277  176.091949       6        1   0.342824
4       69.890198  236.872864       7        1   0.101924
6       67.800011   78.897690       8        1   0.009881
1      -45.106663   19.515642       9        1   0.009881
0       78.090172  -77.689636      10        1   0.009881, topic_info=      Category        Freq           Term       Total  loglift  logprob
3262   Default  101.000000        credito  101.000000  30.0000  30.0000
12058  Default  236.000000      sociedade  236.000000  29.0000  29.0000
6718   Default  232.000000          homem  232.000000  28.0000  28.0000
6754   Default  111.000000         humana  111.000000  27.0000  27.0000
6720   Default  141.000000         homens  141.000000  26.0000  26.0000
...        ...         ...            ...         ...      ...      ...
9946   Topic10    0.000584       politica  112.993460  -2.9505  -9.4589
2013   Topic10    0.000584           caso  142.495555  -3.1832  -9.4597
2823   Topic10    0.000583  consequencias   66.069907  -2.4151  -9.4602
5127   Topic10    0.000583           eram  103.436724  -2.8642  -9.4610
12049  Topic10    0.000581         social  189.860497  -3.4750  -9.4645

[627 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
78         1  0.209180         1933
78         3  0.627539         1933
79         4  0.697562         1934
148        2  1.016014           51
191        1  0.585039  abertamente
...      ...       ...          ...
13063      2  1.022566       valida
13064      1  0.105968     validade
13064      2  0.847745     validade
13241      1  0.997037         vida
13400      1  0.682622        votar

[417 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 10, 4, 6, 3, 8, 5, 7, 2, 1])

In [28]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_ubiratan_jorge_iorio, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  3.7734350821754923 %
Topics found via LDA:

Topic #0:
escolastica la xvii salamanca xvi teologia escolasticos lessio lugo juan

Topic #1:
gps palimpsestos estrela morita hedy admiraveis sussex mazzucato dotes eua

Topic #2:
taxa juros capital producao poupanca governo economia consumo estrutura precos

Topic #3:
extincao baratas animais ameacados baleias ratos molinari pulgas mosquitos moscas

Topic #4:
economia mercado hayek teoria mises capital precos austriaca bens escola

Topic #5:
is that are by austrian on it not with they

Topic #6:
linguagem freire gramsci educacao uber esquerda bakhtin batalha educador jornais

Topic #7:
aprenderam pais presidente estupidez acrescimo fernando deslumbramento debates 127 per

Topic #8:
say moeda monetaria precos dinheiro inflacao consumo producao lei bancos

Topic #9:
mariana rei comercio ravaillac henrique juan consentimento caetano cardeal tirano


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4     -121.098434 -155.305450       1        1  65.336445
2      157.832840  180.367844       2        1  15.246382
8      182.651855   -5.644448       3        1   9.941161
5     -271.352966  -24.664566       4        1   3.639104
0        9.716208  114.812508       5        1   2.480994
9      -36.443634  270.389679       6        1   1.291605
6     -120.992447   28.051109       7        1   0.848240
7       76.950745 -190.080948       8        1   0.729492
1     -197.384354  175.595047       9        1   0.265033
3       19.362465  -41.789871      10        1   0.221544, topic_info=      Category        Freq       Term       Total  loglift  logprob
11662  Default  163.000000        say  163.000000  30.0000  30.0000
8629   Default  231.000000      moeda  231.000000  29.0000  29.0000
12421  Default  157.000000       taxa  157.000000  28.0000  28.0000
7712   Default  200.000000      juros  200.000000  27.0000  27.0000
10356  Default  288.000000   producao  288.000000  26.0000  26.0000
...        ...         ...        ...         ...      ...      ...
972    Topic10    0.522246    animais    4.840135   3.8857  -5.5587
10624  Topic10    0.246442     pulgas    1.946150   4.0458  -6.3097
8640   Topic10    0.253989   molinari    2.204437   3.9514  -6.2795
8721   Topic10    0.210477  mosquitos    3.229159   3.3817  -6.4674
6325   Topic10    0.160885  garantias    1.897227   3.6448  -6.7361

[537 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
19         8  0.894513   127
99         3  0.811571  1800
161        3  0.861941  1964
200        2  0.962278  2017
208        8  0.892982   255
...      ...       ...   ...
13434      1  0.281089   xvi
13434      5  0.562178   xvi
13434      6  0.093696   xvi
13435      1  0.238003  xvii
13435      5  0.714010  xvii

[671 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 9, 6, 1, 10, 7, 8, 2, 4])

In [29]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_gary_north, n_components=10, number_words=10)

get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  4.341357194869209 %
Topics found via LDA:

Topic #0:
confiam corporativos st1metricconverter primadonas st1personname productid solucao esperanca jaz exportando

Topic #1:
imigrantes trump abril maio imigracao suecia assistencialismo assistencialista dia benesses

Topic #2:
dinheiro bancos governo banco divida centrais dia moeda iria reservas

Topic #3:
armas demissoes massa desarmamento keynes keynesiana formula pergunta desemprego drogas

Topic #4:
precos governo pessoas aumento mercado consumidores economia bens havia producao

Topic #5:
st1personname productid discricionarios agregada el salvador alimentos mutuarios apuros dubai

Topic #6:
united youtube iframe passageiro airlines video videos allowfullscreen 315 aerea

Topic #7:
dinheiro united governo bancos monetaria central passageiro iframe airlines politicos

Topic #8:
bancos dinheiro governo pessoas precos banco economia monetaria centrais irao

Topic #9:
bancos governo dinheiro banco monetaria central juros tit

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
9     -113.953514   81.858459       1        1  51.235491
4      -24.846598  -74.626534       2        1  44.904619
3      151.035324 -113.211761       3        1   1.783162
1      159.173035  175.332108       4        1   1.222801
6      -15.063511  207.381531       5        1   0.409360
5     -204.428497  -49.859432       6        1   0.219598
0     -122.060143 -206.706924       7        1   0.191046
8      241.531311   18.483006       8        1   0.011308
2       52.174885 -238.749908       9        1   0.011308
7       61.937588   43.266823      10        1   0.011308, topic_info=     Category        Freq       Term       Total  loglift  logprob
1450  Default  471.000000     bancos  471.000000  30.0000  30.0000
1449  Default  294.000000      banco  294.000000  29.0000  29.0000
7609  Default  255.000000  monetaria  255.000000  28.0000  28.0000
6869  Default  218.000000      juros  218.000000  27.0000  27.0000
3645  Default   87.000000        dia   87.000000  26.0000  26.0000
...       ...         ...        ...         ...      ...      ...
4566  Topic10    0.000640  esperanca   23.869006  -1.4385  -9.3260
7744  Topic10    0.000640    mundial   68.170371  -2.4882  -9.3263
1313  Topic10    0.000639   aumentar   50.321798  -2.1862  -9.3279
8730  Topic10    0.000638   politica  146.623748  -3.2572  -9.3295
8633  Topic10    0.000638    pessoas  215.477789  -3.6422  -9.3295

[668 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
24         1  0.885743     1777
49         3  0.570264     1915
151        1  0.456142       23
151        3  0.228071       23
151        4  0.228071       23
...      ...       ...      ...
11871      2  0.340736      www
11871      5  0.340736      www
11890      2  0.550855  youtube
11890      5  0.367237  youtube
11900      1  0.993217     zona

[456 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 5, 4, 2, 7, 6, 1, 9, 3, 8])

In [30]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_murray_rothbard, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  4.44039404379638 %
Topics found via LDA:

Topic #0:
governo mercado livre left inflacao precos mises credito economia salarios

Topic #1:
comunismo direito responsabilidade livre boetie la propriedade agressao nafta defesa

Topic #2:
strongmito laffer curva cairam deducoes 19821983 evasao previu tradeoff crowdingout

Topic #3:
dinheiro bancos precos bens mercado depressao oferta banco credito demanda

Topic #4:
seca secas chuvas vivenciando reservatorios persistirem racionamentos avidas adoram ordenam

Topic #5:
tzu engels lao pao chien ateismo chuang henry jenny militante

Topic #6:
left mises economia viena socialismo von universidade st1personname edimburgo productid

Topic #7:
left mises bancos mercado governo dinheiro precos depressao bens economia

Topic #8:
governo mercado pessoas livre propriedade fato liberdade sociedade sendo seja

Topic #9:
culpa igualitaristas mulheres sentimento cultura unanimidade buchanan juizos tullock igualdade


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
8     -137.186356   27.577908       1        1  70.415619
3      -37.198040  125.034157       2        1  18.703344
1      172.977707    5.716744       3        1   6.934841
6        7.365560  -74.975838       4        1   1.894654
9      -32.603352   19.022089       5        1   1.221462
5      116.603806 -107.848648       6        1   0.578869
2       68.745872    6.607920       7        1   0.131045
4       83.005455  112.606773       8        1   0.108064
7     -102.336670  -91.369400       9        1   0.006051
0        2.945338 -178.238739      10        1   0.006051, topic_info=      Category        Freq         Term       Total  loglift  logprob
5692   Default  560.000000     dinheiro  560.000000  30.0000  30.0000
10548  Default  152.000000         left  152.000000  29.0000  29.0000
11506  Default  139.000000        mises  139.000000  28.0000  28.0000
6024   Default  332.000000     economia  332.000000  27.0000  27.0000
13514  Default  368.000000       precos  368.000000  26.0000  26.0000
...        ...         ...          ...         ...      ...      ...
16810  Topic10    0.000438       teoria  308.822096  -3.7538  -9.6496
2435   Topic10    0.000433         bens  266.503014  -3.6166  -9.6598
7863   Topic10    0.000427      federal   96.257728  -2.6132  -9.6747
6036   Topic10    0.000428  economistas  149.782838  -3.0533  -9.6726
7822   Topic10    0.000427         fato  311.653207  -3.7871  -9.6738

[598 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
121        2  0.990168       18pt
128        2  0.935945       1913
134        1  0.521002       1920
134        2  0.260501       1920
134        4  0.208401       1920
...      ...       ...        ...
17992      4  0.804006       wien
18009      4  0.858721       word
18012      2  0.986415        wst
18013      6  0.996541  wurmbrand
18075      3  0.824125       zona

[599 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 4, 2, 7, 10, 6, 3, 5, 8, 1])

In [31]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_helio_beltrao, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  3.3719674609459362 %
Topics found via LDA:

Topic #0:
bancos banco mercado central dinheiro anos juros crise madoff governo

Topic #1:
ideias brasil internet paises pessoas governos mises equipe governo hoje

Topic #2:
inmetro mercado lobby selo pirataria campanha interesses garantia privados 20

Topic #3:
palestra imb financeira impostos crise livre sociedade propriedade voluntaria reducao

Topic #4:
liberdade madoff solo brasil anos livre impostos governo investimento fund

Topic #5:
solo greedo forca ameaca han letal pna lucas razoavelmente neutralizar

Topic #6:
trump hillary entendeu senado republicanos escolha apertada eleicao candidatos carolina

Topic #7:
that is we you are it not or one your

Topic #8:
that impostos is inmetro we mercado lobby sociedade distorcao reducao

Topic #9:
governo impostos seja liberdade caso mercado livre sociedade direito brasil


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
9       11.831623  -52.807606       1        1  36.773459
0       57.063660   35.268906       2        1  29.888658
1       77.512840  -24.904558       3        1  24.971217
7      116.727486   46.121773       4        1   5.775253
5        3.662287    8.407496       5        1   1.205471
6      -56.149071   63.744896       6        1   0.640003
2      -56.029709   -7.536925       7        1   0.639102
3       66.723801  103.786964       8        1   0.048954
4        7.154867   68.080925       9        1   0.028942
8       -5.675244  127.374733      10        1   0.028942, topic_info=     Category       Freq       Term       Total  loglift  logprob
4461  Default  75.000000    mercado   75.000000  30.0000  30.0000
3169  Default  20.000000      forca   20.000000  29.0000  29.0000
6641  Default  20.000000       that   20.000000  28.0000  28.0000
4001  Default  20.000000         is   20.000000  27.0000  27.0000
829   Default  51.000000     bancos   51.000000  26.0000  26.0000
...       ...        ...        ...         ...      ...      ...
3569  Topic10   0.000816  igualdade   20.398554  -1.9784  -8.8477
1034  Topic10   0.000816   campanha    3.935276  -0.3330  -8.8479
3396  Topic10   0.000816    governo  105.190226  -3.6193  -8.8483
4642  Topic10   0.000815   nacional   10.635373  -1.3287  -8.8493
7176  Topic10   0.000815       your    6.599464  -0.8516  -8.8494

[611 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
40        5  1.100886   1977
58        1  0.268494     20
58        2  0.469865     20
58        3  0.201371     20
60        2  0.943545   2000
...     ...       ...    ...
7138      4  0.791871  which
7150      4  0.791418   with
7173      3  0.814633  yoani
7175      4  0.991426    you
7176      4  0.909165   your

[409 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 1, 2, 8, 6, 7, 3, 4, 5, 9])

In [32]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_juan_ramon_rallo, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  4.518602738400503 %
Topics found via LDA:

Topic #0:
bitcoin redes sociais crise soros pessoas democracia fiscais politico refugios

Topic #1:
governo impostos gastos divida dinheiro pib economia juros pais grafico

Topic #2:
pib balticos letonia paises saude estonia 1987 quase lituania sera

Topic #3:
estonia balticos letonia lituania alcancado worldeconomicoutlook findthedata org http emprego

Topic #4:
saude eua seguradoras americano apolices medicina medicos socializacao seguros incentivo

Topic #5:
democracia crise voto economia assuntos seja vies conspiratorias bancos mercado

Topic #6:
pobreza capital bens renda pessoas riqueza seja capitalista hoje trabalho

Topic #7:
crise maneira empresarios culpados bens riqueza saude mercado pib economia

Topic #8:
riqueza capitalista trabalhador trabalho bens empreendedor empreendimento entanto producao seja

Topic #9:
empresarios mercado livre muro socialismo interesses socialista empresas alemanha concorrencia


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
1     -20.375195   57.729717       1        1  42.318191
6     -39.498238    5.188825       2        1  41.628059
0     -78.509705   70.153534       3        1   7.227514
9      18.446409  -44.332275       4        1   7.095314
3     -47.785515  -52.006516       5        1   0.983983
4      42.219910   74.188858       6        1   0.672339
7      15.555558   14.948257       7        1   0.018650
8     -97.258141    6.088395       8        1   0.018650
5      72.041336    9.510844       9        1   0.018650
2     -16.812323  114.342514      10        1   0.018650, topic_info=     Category        Freq          Term       Total  loglift  logprob
3732  Default  281.000000       governo  281.000000  30.0000  30.0000
2730  Default   58.000000   empresarios   58.000000  29.0000  29.0000
4798  Default   78.000000       mercado   78.000000  28.0000  28.0000
5591  Default  117.000000       pobreza  117.000000  27.0000  27.0000
4009  Default  134.000000      impostos  134.000000  26.0000  26.0000
...       ...         ...           ...         ...      ...      ...
53    Topic10    0.000754            20   19.786695  -1.5881  -8.8928
4331  Topic10    0.000752  investimento   32.178219  -2.0766  -8.8950
5558  Topic10    0.000752       piketty   38.188446  -2.2484  -8.8956
5592  Topic10    0.000751       podemos   14.380366  -1.2725  -8.8963
63    Topic10    0.000750          2009   11.037581  -1.0101  -8.8985

[725 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
33        4  0.741524                  1950
36        4  0.778595                  1961
41        2  0.986516                  1987
53        1  0.353773                    20
53        2  0.606468                    20
...     ...       ...                   ...
7389      3  0.837664                  vies
7458      2  0.604197               voltada
7494      1  0.357139                  voto
7494      3  0.535708                  voto
7505      5  0.749276  worldeconomicoutlook

[562 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 1, 10, 4, 5, 8, 9, 6, 3])

In [33]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_hans_hermann_hoppe, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  5.723218007503126 %
Topics found via LDA:

Topic #0:
guerra democracia monarquia encaixes austria guerras europa von wilson reis

Topic #1:
propriedade producao meios socialismo capital bens democracia pessoas pessoa privada

Topic #2:
is that this it all from or are people money

Topic #3:
propriedade pessoa social producao pessoas privada bens direitos seja caso

Topic #4:
encaixes democracia dinheiro propriedade bens is ordem left sera producao

Topic #5:
propriedade pessoas pessoa privada democracia seja bens direito social conflitos

Topic #6:
propriedade dinheiro producao pessoas governo bens protecao seguradoras seja estados

Topic #7:
ofertar pesquisa valentoes economizar sofisticados desejaveis insistir beneficiariam fiz burlesco

Topic #8:
propriedade producao is that this privada people all are pessoa

Topic #9:
propriedade producao socialismo meios bens privada precos mercado pessoa usuarios


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6      131.481354   15.362932       1        1  50.410610
5      -10.423822 -121.899414       2        1  45.265058
2       44.418316  -38.477226       3        1   2.463164
0     -111.375542  -74.326401       4        1   1.729295
7     -116.681984   50.988129       5        1   0.094143
9      111.018669 -103.449654       6        1   0.007546
4       25.718416   51.458458       7        1   0.007546
3      -36.796005  124.524055       8        1   0.007546
1       88.674896  120.928596       9        1   0.007546
8      -42.823078   -9.535112      10        1   0.007546, topic_info=      Category        Freq         Term       Total  loglift  logprob
3143   Default  164.000000   democracia  164.000000  30.0000  30.0000
9340   Default  787.000000  propriedade  787.000000  29.0000  29.0000
5621   Default   63.000000       guerra   63.000000  28.0000  28.0000
3621   Default  221.000000     dinheiro  221.000000  27.0000  27.0000
9168   Default  213.000000     producao  213.000000  26.0000  26.0000
...        ...         ...          ...         ...      ...      ...
9139   Topic10    0.000538     problema  121.465741  -2.8361  -9.1365
8928   Topic10    0.000529       precos  103.058085  -2.6879  -9.1526
1628   Topic10    0.000525      capital   77.457652  -2.4104  -9.1607
10448  Topic10    0.000534         seja  255.283767  -3.5855  -9.1432
10624  Topic10    0.000511     situacao   94.472460  -2.6348  -9.1866

[680 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
24         4  1.028113         1917
25         4  0.712680         1918
109        2  0.976098  abertamente
144        3  0.941945        about
201        2  0.977417    academico
...      ...       ...          ...
11974      2  0.682222           xx
11974      4  0.194921           xx
11984      1  0.873306      zelador
11984      4  0.072775      zelador
11985      1  0.904719    zeladores

[470 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 3, 1, 8, 10, 5, 4, 2, 9])

In [34]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_fernando_chiocca, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  4.214226633581472 %
Topics found via LDA:

Topic #0:
is that it with was by this are he his

Topic #1:
pessoas mercado brasil governo anos propriedade casa economia livre seja

Topic #2:
preservacao madeira proprietarios producao futura presente terra geracao ambientalistas cerceadas

Topic #3:
ptbr brasil futebol setor campeoes aposentadoria jogadores copa brasileiros censo

Topic #4:
future present conservation production laws environment timber more resources must

Topic #5:
that is it are an be by they people not

Topic #6:
crimes criminosos that policia policiais criminosas funcoes loja pessoas policial

Topic #7:
macacos amazonia humanos escada nenhum coisas macaco terra argumento seres

Topic #8:
paulo revolucao separatistas independencia estados brasil uruguai brasilia guerra secessao

Topic #9:
jiujitsu helio gracie ciencia economica estilo artes kimura marciais lutador


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1        3.004157   92.656044       1        1  56.016882
0       -9.684464   17.778234       2        1  25.704034
7       64.051147 -101.918861       3        1   6.737696
3      -82.332047   39.947544       4        1   3.673977
9       54.945839  -17.216925       5        1   3.402759
8       -7.609894  -55.688549       6        1   1.960215
4      -25.953375 -127.979156       7        1   1.535476
2       82.600731   55.819004       8        1   0.924837
6      -85.259064  -47.682159       9        1   0.022062
5      127.796158  -33.154995      10        1   0.022062, topic_info=     Category       Freq      Term      Total  loglift  logprob
3665  Default  90.000000        is  90.000000  30.0000  30.0000
6298  Default  73.000000      that  73.000000  29.0000  29.0000
890   Default  51.000000    brasil  51.000000  28.0000  28.0000
3676  Default  58.000000        it  58.000000  27.0000  27.0000
3696  Default  29.000000  jiujitsu  29.000000  26.0000  26.0000
...       ...        ...       ...        ...      ...      ...
6307  Topic10   0.000803     there  22.865588  -1.8381  -8.4786
970   Topic10   0.000794       can  20.297239  -1.7300  -8.4896
3220  Topic10   0.000821        he  37.115064  -2.2995  -8.4556
6783  Topic10   0.000824      with  48.248307  -2.5585  -8.4523
935   Topic10   0.000794       but  32.255320  -2.1927  -8.4891

[569 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
39        6  0.824214   1932
51        5  0.685399   1970
94        4  0.914865    465
194       1  0.998461  acoes
280       2  0.879340  after
...     ...       ...    ...
6771      2  0.775784    why
6771      7  0.258595    why
6775      2  0.977247   will
6783      2  0.994854   with
6798      2  0.997795  would

[456 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 8, 4, 10, 9, 5, 3, 7, 6])

In [35]:
lda_model, data_vectorized, data, lda_output, vectorizer \
    = get_topics(df_text_consolidated_rodrigo_constantino, n_components=10, number_words=10)


get_lda_plot(lda_model, data_vectorized, vectorizer)

Sparsicity:  4.718502418845965 %
Topics found via LDA:

Topic #0:
tapar peneira sol cartel mercado governo caso americano livre impostos

Topic #1:
governo media ipea ricos mercado impostos reducao salarios block juros

Topic #2:
diretor impostos governo imposto empresa saude 30 carga conta seguranca

Topic #3:
governo liberdade direito mises livre impostos pessoas caso violencia block

Topic #4:
juros dubai taxa capital dinheiro investimentos pessoas credito poupanca crise

Topic #5:
chavez militares americano bases petrobras petista estrago justificar colombia latinoamericanos

Topic #6:
estudo reducao ipea media ganham salarios dias trabalhadores metade estatisticas

Topic #7:
programa bolivariano nacional vitoria governo impostos media estudo salarios juros

Topic #8:
cartel mercado livre consumidores caso carteis preco tamanho rothbard firma

Topic #9:
setor privatizacao privatizacoes estatais aco siderurgico pais esquerda milhoes toneladas


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x           y  topics  cluster       Freq
topic                                                   
3     -23.514744  -48.374123       1        1  69.294032
9     -22.633593   71.982033       2        1   8.358146
8      38.191589  104.740082       3        1   8.357323
4     -88.901131   -0.294751       4        1   6.194319
2      43.038532  -23.656025       5        1   4.639637
5     -28.851915   12.837631       6        1   2.925127
7      25.456575   37.074650       7        1   0.078323
0     -85.749962   70.556725       8        1   0.071500
6      85.248207   45.151508       9        1   0.040796
1     -32.690804  131.493958      10        1   0.040796, topic_info=     Category        Freq         Term       Total  loglift  logprob
1737  Default   28.000000     impostos   28.000000  30.0000  30.0000
1601  Default  108.000000      governo  108.000000  29.0000  29.0000
2097  Default   23.000000      mercado   23.000000  28.0000  28.0000
555   Default   14.000000       cartel   14.000000  27.0000  27.0000
2966  Default   17.000000        setor   17.000000  26.0000  26.0000
...       ...         ...          ...         ...      ...      ...
1588  Topic10    0.000931       gestao    6.853742  -1.0992  -8.0739
3168  Topic10    0.000931        total    6.127451  -0.9873  -8.0740
1999  Topic10    0.000931        livro   13.425255  -1.7723  -8.0747
1207  Topic10    0.000930      entanto    5.546025  -0.8893  -8.0757
320   Topic10    0.000929  arrecadados    2.953593  -0.2601  -8.0765

[682 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2         1  0.865736        106
4         2  0.950131         14
8         5  0.801680         17
13        1  0.866857        197
14        2  0.949771       1970
...     ...       ...        ...
3303      1  0.968480  violencia
3303      3  0.053804  violencia
3345      4  1.057361    welfare
3348      4  0.706957      world
3354      3  0.951268        xxi

[509 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 10, 9, 5, 3, 6, 8, 1, 7, 2])